![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**0- Connect to your Google Drive folder**

In [1]:
my_folder = "/content/drive/MyDrive/Concentracion_SAS"


from google.colab import drive
drive.mount('/content/drive')

# Change the following code to set your Drive folder
import os
os.chdir(my_folder)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Concentracion_SAS


**1- Make sure that your Python version is 3.3 or higher as well as your Java version is 1.8.0_162 or higher**

In [2]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [3]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [4]:
pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 5.3 MB/s 
  Created wheel for saspy: filename=saspy-4.4.1-py3-none-any.whl size=9937304 sha256=1e7f32ed072cfc09ffac4777cd63a5a8304d7cb2208727ab3d2085f5a86e448b
  Stored in directory: /root/.cache/pip/wheels/0a/f6/ab/f7173c8a824856c66f6fd50194bb8fd3ef93fb4f1824be8d0c
Successfully built saspy


**3- Create the configuration file "sascfg_personal.py"**
Please, check that your Home Region is correct, you can check it at [ODA-SAS](https://welcome.oda.sas.com/home)

In [5]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 1
#'iomhost' : ['odaws01-usw2.oda.sas.com','odaws02-usw2.oda.sas.com','odaws03-usw2.oda.sas.com','odaws04-usw2.oda.sas.com'],
#US Home Region 2
'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
#European Home Region 1
#'iomhost' : ['odaws01-euw1.oda.sas.com','odaws02-euw1.oda.sas.com'],
#Asia Pacific Home Region 1
#'iomhost' : ['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'],
#Asia Pacific Home Region 2
#'iomhost' : ['odaws01-apse1-2.oda.sas.com','odaws02-apse1-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create your .authinfo**

If there is no .authinfo file, you can create this

In [6]:
#%%writefile .authinfo
#oda user USR password PSW

Copy this file to home

In [7]:
!cp .authinfo ~/.authinfo

**5- Establish Connection (Need to do this step each time you use SASPy)**

In [8]:
import saspy
sas_session = saspy.SASsession(cfgfile=os.path.join(
    my_folder,"sascfg_personal.py"))
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 355



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/Concentracion_SAS/sascfg_personal.py
WORK Path             = /saswork/SAS_workBC260000391C_odaws01-usw2-2.oda.sas.com/SAS_workCDFC0000391C_odaws01-usw2-2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 14620


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [9]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                      Friday, December  2, 2022 04:55:00 PM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                      Friday, December  2, 2022 04:55:00 PM

33         


For local Jupyter

In [10]:
#%%SAS sas_session
#libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [11]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [40]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
186693,2186694.0,MATEO,EVA,1.0,55.0,27100.0,High School,Suburban,5.0,4.0,...,2.0,4.0,60.0,20.0,40.0,1.0,30.0,0.0,1.0,10.0
928430,2928431.0,WAINWRIGHT,PETER,0.0,63.0,20000.0,High School,City,9.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,20.0


In [41]:
data_merge['Recency'].describe()

count    68433.000000
mean         2.750910
std          1.956686
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         10.000000
Name: Recency, dtype: float64

## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [42]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)  
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=0) 
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)  
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=0)  
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)  
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)  

data_merge[['Recency', 'Frequency', 'Seniority', 'TotalGift', 'MinGift','MaxGift', 'GaveLastYear']].isna().sum()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 


In [43]:
data_merge = pd.get_dummies(data_merge, columns = ['Education', 'City'])
data_merge.head()

,ID,LastName,FirstName,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,...,Contact,GaveThisYear,AmtThisYear,Education_Elementary,Education_High School,Education_University / College,City_City,City_Downtown,City_Rural,City_Suburban
3,2000004.0,LEE,MARY,1.0,78.0,23700.0,3.0,0.0,0.0,0.0,...,1.0,1.0,20.0,0,1,0,0,0,1,0
7,2000008.0,STINSON,MICHAEL,0.0,30.0,92700.0,2.0,0.0,0.0,0.0,...,1.0,1.0,30.0,0,0,1,0,0,0,1
11,2000012.0,GUALTIERI,MARIA,1.0,52.0,77600.0,0.0,0.0,0.0,0.0,...,1.0,1.0,20.0,0,1,0,1,0,0,0
16,2000017.0,OLAGUE,DONNA,1.0,50.0,13000.0,0.0,0.0,0.0,0.0,...,1.0,1.0,75.0,0,1,0,0,0,0,1
20,2000021.0,MCCASKILL,DENISE,1.0,39.0,237800.0,5.0,0.0,0.0,0.0,...,1.0,1.0,20.0,0,0,1,0,0,1,0


## Data Partition

In [44]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.5,random_state=5678) # you can change the percentage
train.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,...,Contact,GaveThisYear,AmtThisYear,Education_Elementary,Education_High School,Education_University / College,City_City,City_Downtown,City_Rural,City_Suburban
943031,2943032.0,HOLLOMON,JOHN,0.0,46.0,40500.0,0.0,0.0,0.0,0.0,...,0.0,1.0,40.0,0,1,0,1,0,0,0
650267,2650268.0,NORDSTROM,DONALD,0.0,62.0,73500.0,3.0,0.0,0.0,0.0,...,0.0,1.0,200.0,0,1,0,0,0,0,1
113945,2113946.0,CARRINGTON,LYNN,1.0,35.0,59000.0,9.0,1.0,0.0,0.0,...,1.0,1.0,10.0,0,0,1,0,0,1,0
345671,2345672.0,LALA,KIM,1.0,89.0,15700.0,8.0,2.0,8.0,1.0,...,0.0,1.0,100.0,0,1,0,0,1,0,0
510431,2510432.0,MONTICELLO,SILVIA,1.0,26.0,18800.0,8.0,1.0,3.0,3.0,...,1.0,1.0,20.0,1,0,0,0,0,0,1



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [45]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Woman', 'Age', 'Salary', 'SeniorList', 'NbActivities',
       'Referrals', 'Recency', 'Frequency', 'Seniority', 'TotalGift',
       'MinGift', 'MaxGift', 'GaveLastYear','Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Woman', 'Age', 'Salary', 'SeniorList', 'NbActivities',
       'Referrals', 'Recency', 'Frequency', 'Seniority', 'TotalGift',
       'MinGift', 'MaxGift', 'GaveLastYear','Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[ 57.51021241  16.01207906 126.01700828 ...  23.21348513  67.86741674
  62.51259651]


In [46]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

63.99971226141682
54350.02830760208
233.13092524931582


## Regression Tree Model（Py）

In [47]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Woman', 'Age', 'Salary', 'SeniorList', 'NbActivities',
       'Referrals', 'Recency', 'Frequency', 'Seniority', 'TotalGift',
       'MinGift', 'MaxGift', 'GaveLastYear','Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Woman', 'Age', 'Salary', 'SeniorList', 'NbActivities',
       'Referrals', 'Recency', 'Frequency', 'Seniority', 'TotalGift',
       'MinGift', 'MaxGift', 'GaveLastYear','Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=6, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[ 54.63663457  26.45964511 180.34768212 ...  51.66891841  45.74785572
  66.7563874 ]


In [48]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

63.55139602775634
54514.223775328006
233.4828125908372


In [49]:
sample = train.sample(int(train.shape[0]*0.30)) 
x_sample = sample[['Woman', 'Age', 'Salary', 'SeniorList', 'NbActivities',
       'Referrals', 'Recency', 'Frequency', 'Seniority', 'TotalGift',
       'MinGift', 'MaxGift', 'GaveLastYear', 'Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']]
y_sample = sample['AmtThisYear']

In [52]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
parameters={"splitter":["best","random"],
            "max_depth" : [2,3,4],
           "min_samples_leaf":[2,3,4],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,30,40] }

clf = RandomizedSearchCV(estimator=DecisionTreeRegressor(), param_distributions = parameters,
                               cv = 2, n_iter = 1400, n_jobs=-1)
clf.fit(x_sample, y_sample)


RandomizedSearchCV(cv=2, estimator=DecisionTreeRegressor(), n_iter=1400,
                   n_jobs=-1,
                   param_distributions={'max_depth': [2, 3, 4],
                                        'max_features': ['auto', 'log2', 'sqrt',
                                                         None],
                                        'max_leaf_nodes': [None, 10, 20, 30,
                                                           40],
                                        'min_samples_leaf': [2, 3, 4],
                                        'min_weight_fraction_leaf': [0.1, 0.2,
                                                                     0.3, 0.4],
                                        'splitter': ['best', 'random']})

In [53]:
print('Los mejores parametros son: ', clf.best_params_)

Los mejores parametros son:  {'splitter': 'random', 'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 4, 'max_leaf_nodes': 30, 'max_features': None, 'max_depth': 3}


In [54]:
model= DecisionTreeRegressor(splitter = 'best', min_weight_fraction_leaf= 0.1, min_samples_leaf= 4, max_leaf_nodes= 30, max_features= None, max_depth= 3)
model.fit(X_train,Y_train)
y_preds = model.predict(X_valid)

In [55]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,y_preds))
#MSE
print(metrics.mean_squared_error(Y_valid,y_preds))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,y_preds)))

63.429894341173124
54039.49798195516
232.46397136320965


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


## Scoring New Data

### Prepare data for scoring

In [36]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [56]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_contact[['MinGift']] = scoring_data_contact[['MinGift']].fillna(value=0) 
scoring_data_contact[['Recency']] = scoring_data_contact[['Recency']].fillna(value=0) 
scoring_data_contact[['Frequency']] = scoring_data_contact[['Frequency']].fillna(value=0)  
scoring_data_contact[['Seniority']] = scoring_data_contact[['Seniority']].fillna(value=0)  
scoring_data_contact[['TotalGift']] = scoring_data_contact[['TotalGift']].fillna(value=0)  
scoring_data_contact[['MaxGift']] = scoring_data_contact[['MaxGift']].fillna(value=0) 
scoring_data_contact = pd.get_dummies(scoring_data_contact, columns = ['Education', 'City']) 
#scoring_data_contact.head()

X = scoring_data_contact[['Woman', 'Age', 'Salary', 'SeniorList', 'NbActivities',
       'Referrals', 'Recency', 'Frequency', 'Seniority', 'TotalGift',
       'MinGift', 'MaxGift', 'GaveLastYear','Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']] 

regr_predict_contact=model.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

,ID,AmtContact
0,2000001.0,57.286638
1,2000002.0,60.534713
2,2000003.0,131.896494
3,2000004.0,45.425207
4,2000005.0,131.896494


### Predict 'amount given' for members who were not contacted

In [57]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_nocontact[['MinGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)
scoring_data_nocontact[['Recency']] = scoring_data_nocontact[['Recency']].fillna(value=0) 
scoring_data_nocontact[['Frequency']] = scoring_data_nocontact[['Frequency']].fillna(value=0)  
scoring_data_nocontact[['Seniority']] = scoring_data_nocontact[['Seniority']].fillna(value=0)  
scoring_data_nocontact[['TotalGift']] = scoring_data_nocontact[['TotalGift']].fillna(value=0)  
scoring_data_nocontact[['MaxGift']] = scoring_data_nocontact[['MaxGift']].fillna(value=0) 
scoring_data_nocontact = pd.get_dummies(scoring_data_nocontact, columns = ['Education', 'City']) 
#scoring_data_nocontact.head()

X = scoring_data_nocontact[['Woman', 'Age', 'Salary', 'SeniorList', 'NbActivities',
       'Referrals', 'Recency', 'Frequency', 'Seniority', 'TotalGift',
       'MinGift', 'MaxGift', 'GaveLastYear','Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']] 

regr_predict_nocontact=model.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,AmtNoContact
0,2000001.0,57.286638
1,2000002.0,60.534713
2,2000003.0,131.896494
3,2000004.0,45.425207
4,2000005.0,131.896494


In [58]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,57.286638,57.286638
1,2000002.0,60.534713,60.534713
2,2000003.0,131.896494,131.896494


## Exporting Results to a CSV File

In [59]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.

In [ ]:
!head Round2_Output_amt.csv

ID,AmtContact,AmtNoContact
2000001.0,62.05621301775148,62.05621301775148
2000002.0,66.39552464469307,66.39552464469307
2000003.0,65.46658259773014,65.46658259773014
2000004.0,36.37507314218841,36.37507314218841
2000005.0,65.46658259773014,65.46658259773014
2000006.0,33.17251461988304,33.17251461988304
2000007.0,26.45964510589582,26.45964510589582
2000008.0,54.63663457129757,54.63663457129757
2000009.0,39.13897280966767,39.13897280966767
